# Data processing and merging into final results

## Processing NCBI's *Latrodectus* mRNA data

First, let's join id and descriptions for each record in the 'mrna_latrodectus_ncbi.fasta' file. This makes interpretation of the blast searches against this database more stratightforward.


In [28]:
!./add_underscore_fasta_description.py mrna_latrodectus_ncbi.fasta > mrna_latrodectus_underscore.fasta

Removing duplicate sequences using [cd-hit](https://github.com/weizhongli/cdhit/wiki):

In [29]:
!./cd-hit -c 1 -i mrna_latrodectus_underscore.fasta -o mrna_latrodectus_underscore_unique.fasta

Program: CD-HIT, V4.8.1 (+OpenMP), Apr 21 2020, 13:59:26
Command: ./cd-hit -c 1 -i mrna_latrodectus_underscore.fasta -o
         mrna_latrodectus_underscore_unique.fasta

Started: Tue Apr 21 15:16:27 2020
                            Output                              
----------------------------------------------------------------
total seq: 1447
longest and shortest : 5408 and 24
Total letters: 986122
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 1M
Buffer          : 1 X 11M = 11M
Table           : 1 X 65M = 65M
Miscellaneous   : 0M
Total           : 78M

Table limit with the given memory limit:
Max number of representatives: 763452
Max number of word counting entries: 90221755

comparing sequences from          0  to       1447
.
     1447  finished       1403  clusters

Approximated maximum memory consumption: 82M
writing new database
writing clustering information
program completed !

Total CPU time 1.31


<br>

---

Generating translated ORF's for mRNA dataset using **Transdecoder** - Command line used: 

```TransDecoder.LongOrfs -m 30 --output_dir mrna_latrod_transcripts -t mrna_latrodectus_underscore_unique.fasta```

The path to the file with the translated sequences is `./mrna_latrod_transcripts/longest_orfs.pep`

## Blast searches against mRNA

Creating blastable databases for both the nucleotide (MRNA_LATROD_NT) and aa (MRNA_LATROD_AA) sequences:

In [3]:
!makeblastdb -in mrna_latrodectus_underscore_unique.fasta -dbtype nucl -out MRNA_LATROD_NT
!makeblastdb -in ./mrna_latrod_transcripts/longest_orfs.pep -dbtype prot -out MRNA_LATROD_AA



Building a new DB, current time: 04/21/2020 16:11:34
New DB name:   /home/gabriel/annotation_final_results/MRNA_LATROD_NT
New DB title:  mrna_latrodectus_underscore_unique.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1403 sequences in 0.115397 seconds.


Building a new DB, current time: 04/21/2020 16:11:34
New DB name:   /home/gabriel/annotation_final_results/MRNA_LATROD_AA
New DB title:  ./mrna_latrod_transcripts/longest_orfs.pep
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 6132 sequences in 0.402356 seconds.


---
Running Blast searches:

* Blastn - Trinity contigs against MRNA_LATROD_NT
* Blastx - Trinity contigs against MRNA_LATROD_AA
* Blastp - Translated ORF's from Trinity contigs (generated previously by Trinotate) against MRNA_LATROD_AA

In [ ]:
!blastn -db MRNA_LATROD_NT -evalue 1e-5 -query Trinity.120.fasta -out trinity_vs_mrna_blastn.outfmt6 -outfmt 6
!blastx -db MRNA_LATROD_AA -evalue 1e-5 -query Trinity.120.fasta -out trinity_vs_mrna_blastx.outfmt6 -outfmt 6
!blastp -db MRNA_LATROD_AA -evalue 1e-5 -query Trinity.120.pep -out trinity_vs_mrna_blastp.outfmt6 -outfmt 6

## Integrating BlastX and BlastP hits into Trinotate database - Trinotate final report

Adding custom blastx and blastp results to the Trinotate sqlite database:


`Trinotate Trinotate-arthropoda-updated.sqlite LOAD_custom_blast --outfmt6 trinity_vs_mrna_blastx.outfmt6 --prog blastx --dbtype MRNA_LATROD_AA`

`Trinotate Trinotate-arthropoda-updated.sqlite LOAD_custom_blast --outfmt6 trinity_vs_mrna_blastp.outfmt6 --prog blastp --dbtype MRNA_LATROD_AA`

Generating a report based on the updated database - `Trinotate_final_report.xls`:

`Trinotate Trinotate-arthropoda-updated.sqlite report --incl_trans --incl_pep > Trinotate_final_report.xls`

**OBS:** The Blastn results will be incorporated to other analysis later in this report

<br>

---
To further integrate our results, let's import some required libraries:

In [14]:
import pandas as pd
from Bio import SearchIO

We have converted the Trinotate final report from .xls to xlsx in [LibreOffice](https://www.libreoffice.org/) v6.3.5.2 to help integrate the results

## Merging Trinotate and custom_blast hits

Now, we will create dataframes for both our results:

In [15]:
trinotate = pd.read_excel('Trinotate_final_report.xlsx', sep='\t', na_values='.')
custom_blast = pd.read_excel('expressionCount.trinity.120.WholeTable.xlsx')

Let's rename trinotate's first column in order to match it to the corresponding column in custom_blast:

In [16]:
trinotate = trinotate.rename(columns={'#gene_id': 'gene_id'})

'gene_id' and 'Transcript_id' have the same info in both dataframes, so we will use them in an full outer join of these dataframes:

In [17]:
combined_data = pd.merge(trinotate, custom_blast, on=["gene_id", "transcript_id"], how='outer')

## Adding blastn hits against genbank's *Latrodectus* mrna sequences to final dataframe

First, let's parse the Blastn results. We will need to extract the query's gene and transcript id, as well as a concatenated subject_id and e-value:

In [19]:
def parse_blastn(blast_outfmt6):
    hits = SearchIO.parse(blast_outfmt6, 'blast-tab')
    for hit in hits:
        gene_id = hit.id.rsplit('_', 1)[0] #Converting transcript_id to gene_id
        transcript_id = hit.id
        blastn_hits = ";".join(["{}^evalue_{}".format(hsp.hit_id, hsp.evalue) for hsp in hit.hsps]) #Joins all hits for a given transcript using ';'. Subject_id and e-value separated by a '^'
        yield(gene_id, transcript_id, blastn_hits)
    

# OBS: Extraction of info from HSP (High-Socring region) class:

# for hsp in hit.hsps:
#     print(dir(hsp))
#     print(hsp.attribute or hsp.method())

Now, we need to create a dataframe with the parsed blastn info...

In [ ]:
custom_latrod = pd.DataFrame(parse_blastn('trinity_vs_mrna_blastn.outfmt6'), columns = ('gene_id', 'transcript_id', 'blastn_hits'))

And merge this dataframe to the 'combined_data' dataframe:

In [22]:
final_data = pd.merge(combined_data, custom_latrod, on=["gene_id", "transcript_id"], how='outer')

## Merging all data into a final table

Finally, we need to output the final merged results to a excel table:

In [23]:
final_data.to_excel("FINAL_ANNOTATION_RESULTS.xlsx", index=False)